In [1]:
%load_ext autoreload
%autoreload 2
from yahpo_train.cont_normalization import ContNormalization
from yahpo_train.model  import *
from yahpo_gym import cfg
from yahpo_train.metrics import *
from yahpo_gym.benchmarks import lcbench

In [2]:
cfg = cfg("lcbench")
dls = dl_from_config(cfg, bs=2048)

In [3]:
f = FFSurrogateModel(dls, layers=[512,512], deeper = [512,128,128])
l = SurrogateTabularLearner(dls, f, loss_func=nn.MSELoss(reduction='mean'), metrics=nn.MSELoss)
l.metrics = [AvgTfedMetric(mae),  AvgTfedMetric(r2), AvgTfedMetric(spearman)]
l.add_cb(MixHandler)
l.add_cb(EarlyStoppingCallback(patience=3))

In [4]:
l.fit_flat_cos(5, 1e-3)
for p in l.model.wide.parameters():
    p.requires_grad = False
l.fit_flat_cos(5, 1e-4)

epoch,train_loss,valid_loss,mae,r2,spearman,time
0,0.004160,0.003937,[6.48044986e+01 5.75140412e+00 1.50738939e-01 4.79962996e-02 4.85670052e-02 4.84354560e-02],[0.80576631 0.8315823 0.85315017 0.87518378 0.99306039 0.87277413],[0.98030968 0.91779388 0.93585193 0.93259366 0.97086897 0.93161082],03:37
1,0.003446,0.003726,[5.13670233e+01 5.52026622e+00 1.40497463e-01 4.50382610e-02 4.63995355e-02 4.54363523e-02],[0.86548078 0.83460815 0.86726858 0.88118461 0.99268153 0.87942916],[0.99129368 0.91543717 0.93405032 0.93257718 0.97119316 0.93082726],03:35
2,0.003225,0.003657,[4.85240137e+01 5.37603898e+00 1.25048758e-01 4.66420864e-02 4.26947615e-02 4.58026851e-02],[0.87518461 0.84243405 0.8773968 0.87827835 0.99337025 0.87808247],[0.99188564 0.92340314 0.94661319 0.9391233 0.97638391 0.93755188],03:37


KeyboardInterrupt: 

In [6]:
l.export_onnx(cfg)

c:\Users\flo\Documents\yahpo_gym\yahpo_train\yahpo_train\cont_normalization.py:48: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if torch.abs(lmbda) < self.eps:
c:\Users\flo\Documents\yahpo_gym\yahpo_train\yahpo_train\cont_normalization.py:54: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if torch.abs(lmbda - 2.) <= self.eps:
c:\Users\flo\Documents\yahpo_gym\yahpo_train\yahpo_train\model.py:126: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value wil

0.003657344728708267